# Q2.1 - Systematic Encoding Matrix  $G$ 

In [7]:
import numpy as np

def rref(M):
    """
    Computes the Row Reduced Echelon Form (RREF) of matrix M using XOR.
    Returns the RREF matrix and the list of pivot column indices.
    """
    M = M.copy()
    rows, cols = M.shape
    pivot_cols = []
    current_row = 0
    
    for j in range(cols):
        if current_row >= rows:
            break
        
        pivot_row = -1
        for i in range(current_row, rows):
            if M[i, j] == 1:
                pivot_row = i
                break
        
        if pivot_row != -1:
            if pivot_row != current_row:
                M[[current_row, pivot_row]] = M[[pivot_row, current_row]]
            # Record the pivot column
            pivot_cols.append(j)
            
            # Use XOR to zero out all other 1s in column
            for i in range(rows):
                if i != current_row and M[i, j] == 1:
                    # M[i] = M[i] XOR M[current_row]
                    M[i] = M[i] ^ M[current_row]
            
            current_row += 1
            
    return M, pivot_cols

def build_systematic_encoder(H_original):
    """
    Builds the systematic LDPC encoder G using the procedure H_hat = [P | I] and G = [I_K / P].
    """
    H = H_original.copy()
    M, N = H.shape
    K = N - M  # K = message length
    
    # Compute RREF of H to find pivots
    H_rref, pivot_cols = rref(H)

    all_cols = list(range(N))
    non_pivot_cols = [c for c in all_cols if c not in pivot_cols]
    
    # Determine Permutation Order: [Non-Pivots] (P-part) + [Pivots] (I-part)
    perm_order = non_pivot_cols + pivot_cols
    
    # Apply Permutation to the RREF matrix to get the final H_hat = [P | I]
    H_hat = H_rref[:, perm_order]

    P = H_hat[:, :K]  # P is the first K columns of H_hat
    I_K = np.eye(K, dtype=int)
    G = np.vstack([I_K, P])
    
    return H_hat, G


# Test Input
H_matrix = np.array([
    [1, 1, 1, 1, 0, 0],
    [0, 0, 1, 1, 0, 1],
    [1, 0, 0, 1, 1, 0]
], dtype=int)

print("Original H:")
print(H_matrix)

# Run the function
H_hat, G = build_systematic_encoder(H_matrix)

print("-" * 40)
print("System Parameters: N=6, M=3, K=3")
print("-" * 40)

print("\nPermuted and Row-Reduced H (H_hat = [P | I]):")
print(H_hat)

print("\nSystematic Generator Matrix G (G = [I_K / P]):")
print(G)

# Verification: H_hat * G should be 0
check = (H_hat @ G) % 2
print("\nVerification (H_hat @ G % 2):")
print(check)
print(f"All zero? {np.all(check == 0)}")

Original H:
[[1 1 1 1 0 0]
 [0 0 1 1 0 1]
 [1 0 0 1 1 0]]
----------------------------------------
System Parameters: N=6, M=3, K=3
----------------------------------------

Permuted and Row-Reduced H (H_hat = [P | I]):
[[1 1 0 1 0 0]
 [1 1 1 0 1 0]
 [1 0 1 0 0 1]]

Systematic Generator Matrix G (G = [I_K / P]):
[[1 0 0]
 [0 1 0]
 [0 0 1]
 [1 1 0]
 [1 1 1]
 [1 0 1]]

Verification (H_hat @ G % 2):
[[0 0 0]
 [0 0 0]
 [0 0 0]]
All zero? True


---
We must ensure $H$ is in RREF form ($H_{RREF} = [ P \ | \ I ]$) to solve the system of linear equations. This explicitly separates the message bits from the parity bits, allowing us to directly construct $G$ without doing any further algebra. Example $H$ in the question is not in RREF. So, we have implemented a function to convert $H$ to $H_{RREF}$

We verify this orthogonality (that $H_{RREF} \times G = 0$) below:

$H_{RREF} = [ P \ | \ I ]$

$H_{RREF} \times G = [ P \ | \ I ] \times \begin{bmatrix} I \\ P \end{bmatrix}$

   $= (P \times I) + (I \times P)$

   $= P + P = 0$ (since we are operating in binary where $x+x=0$)

we must have our $\hat{H}$ be equal to $H_{RREF}$



# Q2.2 - Factor Graph for Matrix $H$

<img src="Images/factorGraph.png" width="400">

Here is the factor graph from the H given in Question 2.1

A factor graph for an LDPC code is a bipartite graph

Each check node $c_i$ connects to variable node $x_j$ if the corresponding entry in the matrix row is 1 ($H_{ij} = 1$).

$P(x) \propto \psi_1(x_1, x_2, x_3, x_4) \cdot \psi_2(x_3, x_4, x_6) \cdot \psi_3(x_1, x_4, x_5)$,

$
H =
\begin{bmatrix}
1 & 1 & 1 & 1 & 0 & 0 \\
0 & 0 & 1 & 1 & 0 & 1 \\
1 & 0 & 0 & 1 & 1 & 0
\end{bmatrix}
$


# Q2.3 - LDPC decoder

The Sum-Product algorithm is implemented with Log-Likelihood Ratios (LLR) defined as $L = \ln \frac{P(x=0)}{P(x=1)}$.
- $L > 0$ implies the bit is likely 0, and $L < 0$ implies the bit is likely 1.
- The LLR channel for each bit $y_j$ is: $L_j^{channel} = (1 - 2y_j) \ln\left(\frac{1-p}{p}\right)$
- The term $(1- 2y_j)$ ensures the sign is correct: positive if $y_j=0$ and negative if $y_j=1$.

Since we are using positive and negative values, we implement $\tanh$ for our updates.

### Iterative Message Passing

The decoding process iterates between two types of updates on the factor graph until convergence or a maximum iteration count is reached

#### Check Node Update (Horizontal Step)
The check nodes calculate the probability that a parity constraint is satisfied. In the LLR domain, the parity operation (XOR) is transformed into a product of hyperbolic tangents. 
For a check node $i$ connected to variable nodes $j$, the message $r_{i \to j}$ is updated as: 

$r_{i \to j} = 2 \tanh^{-1}\left( \prod_{k \in \mathcal{N}(i) \setminus j} \tanh\left(\frac{q_{k \to i}}{2}\right) \right)$

This formula effectively calculates the "soft XOR" of all other connected bits. If the product is positive, the parity suggests bit $j$ should be 0; if negative, it suggests 1.

#### Variable Node Update (Vertical Step)
The variable nodes aggregate information from all connected check nodes and the original channel observation. Since LLRs are additive, this step is a simple summation:

$q_{j \to i} = L_j^{channel} + \sum_{k \in \mathcal{M}(j) \setminus i} r_{k \to j}$

#### Tentative Decoding and Termination
At the end of each iteration, we compute the total belief $L^{total}_j = L_j^{channel} + \sum_{k} r_{k \to j}$. We estimate $\hat{x}$ based on the sign of $L^{total}$ and check if the syndrome $H\hat{x} = 0$ is satisfied. 

If satisfied, we terminate early.

---

## Optimization
To handle matrix operations efficiently, we avoided standard python loops, which run in $O(E)$, where $E$ is the number of edges. Instead, we used Numpy Vectorization

Matrix-Based Updates: We maintain message matrices $Q$ and $R$ of size $M \times N$. Using boolean masking (mask = H == 1), we update all active edges simultaneously using broadcasted operations.

The check node update requires the product of all neighbors except the target variable ($\prod_{k \neq j}$). Calculating this individually for every edge is computationally expensive.
- We compute the product of the entire row once and divide by the specific element for each edge:  

    $\text{Product}_{\text{excluding } j} = \frac{\text{Total Row Product}}{\text{Element}_j}$

*Note: We included a small epsilon to prevent division-by-zero errors during this step.*



In [ ]:
import numpy as np

def ldpc_decoder(H, y, p, max_iter=20):
    """
    Decodes a received LDPC codeword using Loopy Belief Propagation (Sum-Product).
    
    Args:
        H (np.array): Parity check matrix (M x N).
        y (np.array): Received vector (size N).
        p (float): Noise ratio (probability of flip).
        max_iter (int): Maximum number of iterations.
        
    Returns:
        x_hat (np.array): Decoded vector.
        status (int): 0 for success, 1 for max iterations reached.
        iterations (int): Number of iterations performed.
    """
    M, N = H.shape
    
    # Initialization of LLR channel
    if p == 0: 
        channel_llr = (1 - 2 * y) * 1e10 
    else:
        channel_llr = (1 - 2 * y) * np.log((1 - p) / p)
    
    # Initialize messages 
    Q = np.zeros((M, N))
    R = np.zeros((M, N))
    mask = (H == 1)
    Q[mask] = np.tile(channel_llr, (M, 1))[mask]
    
    
    for iteration in range(max_iter):
        # Horizontal Step: Check Node Updates
        tanh_Q = np.tanh(Q / 2.0)
        tanh_Q = np.clip(tanh_Q, -0.999999, 0.999999)    # Clip for numerical stability
        
        # Optimized product-excluding-self calculation
        safe_tanh = np.where(mask, tanh_Q, 1.0)
        row_prod = np.prod(safe_tanh, axis=1)[:, np.newaxis]
        denom = safe_tanh + 1e-20 
        prod_exclude_self = row_prod / denom
        
        R[mask] = (2 * np.arctanh(prod_exclude_self))[mask]
        
        # Vertical Step: Variable Node Updates
        total_R_sum = np.sum(R, axis=0)
        Q[mask] = (channel_llr + total_R_sum - R)[mask]
        
        # Tentative Decoding
        L_total = channel_llr + total_R_sum
        x_hat = (L_total < 0).astype(int)
        
        # Check Syndrome
        syndrome = H @ x_hat % 2
        if not np.any(syndrome):
            return x_hat, 0, iteration + 1
            
    return x_hat, -1, max_iter

H1 = np.loadtxt('Data/H1.txt', dtype=int)
y1 = np.loadtxt('Data/y1.txt', dtype=int)
p = 0.1

decoded_msg, status, iters = ldpc_decoder(H1, y1, p)

print(f"Decoding Status: {'Success' if status == -1 else 'Failed'}")
print(f"Iterations: {iters}")
print(f"Decoded Vector: {decoded_msg}")

Decoding Status: Success
Iterations: 8
Decoded Vector: [0 1 0 0 1 0 0 0 0 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 1 1 1 0 0 0 0 0 1 1 1 1
 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 1 1 1 1 0 1 1 0 1 1 0 0 0 1
 1 0 1 0 0 1 0 1 1 0 0 1 0 0 0 1 1 0 0 0 0 1 0 1 1 1 1 0 0 1 0 1 1 1 0 0 1
 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 1 1 0 1 0 1 1 0
 1 0 0 1 0 1 1 1 0 1 0 0 0 1 1 1 0 0 1 0 0 1 1 1 1 0 0 1 0 0 1 0 0 1 1 0 0
 1 0 0 0 1 0 0 0 1 1 0 0 0 0 1 0 1 1 1 0 1 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 1
 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 1 1 0 0 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 1 0 1 0 1 1 0 0 1 1 0 1 0
 1 0 1 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 1 1 0 0 1
 0 1 0 1 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 1 1 0 1 0 1 0 1
 1 0 0 1 1 0 0 1 0 1 0 0 1 1 1 0 1 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 1 1
 0 0 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 1 0 0
 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 0 0 1 1 0 1 

**Convergence**: It successfully converged in 8 iterations.

**Outcome**: The syndrome check confirmed $Hx = 0$.

# Q3.4 Decode Message

To retrieve the message, the first 248 characters are iterated through, with every 8 bits being converted into ASCII

Each ASCII character is then appened into a list and returned as a string once all 248 bits have been iterated through.

In [11]:
def decode_message(decoded_vector):
    """
    Recovers the original English message from the decoded LDPC vector.
    
    Args:
        decoded_vector (np.array): The full decoded output from the LDPC decoder.
        
    Returns:
        str: The recovered ASCII string.
    """
    message_bits = decoded_vector[:248].astype(int)
    chars = []
    for i in range(0, 248, 8):
        byte_chunk = message_bits[i : i+8]
        char_code = 0
        for bit in byte_chunk:
            char_code = (char_code << 1) | bit
        chars.append(chr(char_code))
        
    return "".join(chars)


if status == 0:
    secret_message = decode_message(decoded_msg)
    print(f"Original Message: {secret_message}")
else:
    print("Decoding failed, cannot recover message.")

Original Message: Happy Holidays! Dmitry&David :)
